In [2]:
import time
import urllib2
import xml.dom.minidom

In [10]:
dom = xml.dom.minidom.parseString('<data><rec>Hello!</rec></data>')

In [11]:
dom

<xml.dom.minidom.Document instance at 0x064FA698>

In [12]:
r = dom.getElementsByTagName('rec')

In [13]:
r

[<DOM Element: rec at 0x64fa620>]

In [14]:
r[0].firstChild

<DOM Text node "u'Hello!'">

In [15]:
r[0].firstChild.data

u'Hello!'

In [16]:
kayakkey='YOUR KEY HERE'

def getkayaksession():
    # Construct the URL to start a session
    url='http://www.kayak.com/k/ident/apisession?token=%s&version=1' % kayakkey

    # Parse the resulting XML
    doc=xml.dom.minidom.parseString(urllib2.urlopen(url).read())

    # Find <sid>xxxxxxxx</sid>
    sid=doc.getElementsByTagName('sid')[0].firstChild.data
    return sid

In [17]:
sid = getkayaksession()

AttributeError: 'NoneType' object has no attribute 'data'

In [18]:
def flightsearch(sid,origin,destination,depart_date):
  
    # Construct search URL
    url='http://www.kayak.com/s/apisearch?basicmode=true&oneway=y&origin=%s' % origin
    url+='&destination=%s&depart_date=%s' % (destination,depart_date)
    url+='&return_date=none&depart_time=a&return_time=a'
    url+='&travelers=1&cabin=e&action=doFlights&apimode=1'
    url+='&_sid_=%s&version=1' % (sid)

    # Get the XML
    doc=xml.dom.minidom.parseString(urllib2.urlopen(url).read())

    # Extract the search ID
    searchid=doc.getElementsByTagName('searchid')[0].firstChild.data

    return searchid

In [19]:
def flightsearchresults(sid,searchid):
    def parseprice(p): 
        return float(p[1:].replace(',',''))

    # Polling loop
    while 1:
        time.sleep(2)

        # Construct URL for polling
        url='http://www.kayak.com/s/basic/flight?'
        url+='searchid=%s&c=5&apimode=1&_sid_=%s&version=1' % (searchid,sid)
        doc=xml.dom.minidom.parseString(urllib2.urlopen(url).read())

        # Look for morepending tag, and wait until it is no longer true
        morepending=doc.getElementsByTagName('morepending')[0].firstChild
        if morepending==None or morepending.data=='false': break

    # Now download the complete list
    url='http://www.kayak.com/s/basic/flight?'
    url+='searchid=%s&c=999&apimode=1&_sid_=%s&version=1' % (searchid,sid)
    doc=xml.dom.minidom.parseString(urllib2.urlopen(url).read())

    # Get the various elements as lists
    prices=doc.getElementsByTagName('price')
    departures=doc.getElementsByTagName('depart')
    arrivals=doc.getElementsByTagName('arrive')  

    # Zip them together
    return zip([p.firstChild.data.split(' ')[1] for p in departures],
                [p.firstChild.data.split(' ')[1] for p in arrivals],
                [parseprice(p.firstChild.data) for p in prices])

In [20]:
sid=getkayaksession()
searchid=flightsearch(sid,'BOS','LGA','11/17/2006')
f=flightsearchresults(sid,searchid)

AttributeError: 'NoneType' object has no attribute 'data'

In [21]:
def createschedule(people,dest,dep,ret):
    # Get a session id for these searches
    sid=getkayaksession()
    flights={}
  
    for p in people:
        name,origin=p
        # Outbound flight
        searchid=flightsearch(sid,origin,dest,dep)
        flights[(origin,dest)]=flightsearchresults(sid,searchid)

        # Return flight
        searchid=flightsearch(sid,dest,origin,ret)
        flights[(dest,origin)]=flightsearchresults(sid,searchid)
    
    return flights

In [22]:
f = createschedule(optimization.people[0:2],'LGA','11/17/2006','11/19/2006')
optimization.flight=f
domain = [(O,30)] * len(f)
s = optimization.geneticoptimize(domain,optimization.schedulecost)
printschedule(s)

NameError: name 'optimization' is not defined